In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

donnees = pd.read_csv("/kaggle/input/metacritic-all-time-games-stats/metacritic_games.csv")
donnees["index"] = pd.DataFrame({"index":range(0, 20423)})
donnees = donnees.set_index("index")
pd.options.mode.chained_assignment = None
donnees.head()



## Description des colonnes
### Name : nom du jeu vidéo
### Platform : PC / PS / N64 ...
### Developer
### Publisher
### genre
### players : nombre de joueurs (local and online)
### rating : note ESRB(organisme qui évalue les jeux vidéos pour voir à qui ils conviennent)
### attribute : Information supplémentaire sur le jeu vidéo
### release_date : Date de sortie du jeu vidéo
### link : Lien internet du jeu vidéo
### critic_positive : nombre de critique positive
### critic_neutral : nombre de critique neutre
### critic_negative : nombre de critique negative
### metascore : note moyenne des critiques
### user_positive : nombre de critiques positives par les user
### user_neutral : nombre de critiques neutre par les user
### user_negative : nombre de critiques negatives par les user
### user_score : note moyenne des utilisateurs

 ## Attributs du fichier

In [ ]:
donnees.shape

In [ ]:
donnees.columns

## Gestion des NaN

In [ ]:
donnees.isnull().sum()

#### Certaines colonnes possèdent des informations manquantes :
#### Attribute : information supplémentaire sur le jeu vidéo est rempli de quasiment que des valeurs NaN, il serait donc judicieux de la supprimer, de plus elle n'apporte pas d'informations intéressante
#### players  : Supprimer la colonne players car les informations sont discornues.

## Suppression des colonnes inutiles pour le projet

In [ ]:
donnees = donnees.drop(columns=["attribute", "link", "players"])
donnees.head()

## Gestion des données dupliquées

In [ ]:
donnees[donnees.duplicated()].shape

In [ ]:
donnees[donnees.duplicated()]

In [ ]:
donnees[donnees["name"] == "Bookworm"]

In [ ]:
donnees = donnees.drop_duplicates()
donnees[donnees["name"] == "Bookworm"]

#### Les données ont bien été supprimées

## Exploration du fichier

In [ ]:
donnees.dtypes

#### Transformer la colonne release_date en date

In [ ]:
donnees["release_date"] = pd.to_datetime(donnees["release_date"])

In [ ]:
donnees.dtypes

#### Créer une colonne Year qui contiendra seulement l'année de sortie du jeu vidéo

In [ ]:
donnees["year"] = donnees["release_date"].dt.year
donnees.columns

### Evolution des plateforme de jeu au cours du temps

In [ ]:
# Quelles sont les plateformes les plus prisées ?
donnees["platform"].value_counts().head()

In [ ]:
data_plat = donnees[(donnees["platform"] == "PS4") | (donnees["platform"] == "PC") | (donnees["platform"] == "XONE")]
data_platform = data_plat.groupby(["platform", "year"]).size().reset_index(name = "Count")

In [ ]:
plt.figure(figsize=[15, 6])
plt.subplot(111)
sns.pointplot(x = "year", y = "Count", data = data_platform, hue = "platform")
plt.title("Evolution des 3 plus grande plateformes de ces 8 dernières années au cours du temps")
plt.xticks(rotation = 90)
plt.show()

#### La PS4 est sorti en novembre 2013 et a concurrencé le PC pendant 5 années.
#### La même année est sorti la Xbox One venu succéder à la Xbox360
#### On constate, aussi, que le PC reste quasiment toujours en tête des videos gamers.

### 5 plus grandes plateformes de jeu

In [ ]:
top_platform = donnees["platform"].value_counts().head()
data_top_platform = donnees[donnees.isin(top_platform.index)]

In [ ]:
palette = sns.color_palette("Paired")
sns.set_palette(palette)
plt.figure(figsize=[15, 6])
plt.subplot(111)
sns.countplot(data = data_top_platform, x = "platform", order = data_top_platform["platform"].value_counts().keys())
plt.title("Top 5 des plateformes de jeu vidéos")
plt.xticks(rotation = 90)
plt.show()

#### On voit que le PC reste la plateforme de jeu la plus prisée, et ceci même historiquement.

### Analyse des notes ESRB

In [ ]:
top_rating = donnees["rating"].value_counts()
df_rating = donnees[donnees.isin(top_rating.index)]
top_rating

In [ ]:
palette = sns.color_palette("Paired")
sns.set_palette(palette)
plt.figure(figsize=[15, 6])
plt.subplot(111)
sns.countplot(data = df_rating, x = "rating", order = df_rating["rating"].value_counts().keys())
plt.title("Classement des jeux vidéos suivant leurs note ESRB (rating)")
plt.xticks(rotation = 90)
plt.show()

#### La catégorie Enfant / Adulte (E) est la catégorie de jeu la plus représentée, suivi de très près par la catégorie T dont le contenu convient aux enfants de plus de 13 ans (teenage).

### Top des sociétes développeurs de Jeux Vidéos

In [ ]:
donnees_dev = donnees["developer"].value_counts().head()
data_dev = donnees[donnees.isin(donnees_dev.index)]

In [ ]:
palette = sns.color_palette("Paired")
sns.set_palette(palette)
plt.figure(figsize=[15, 6])
plt.subplot(111)
sns.countplot(data = data_dev, x = "developer", order = data_dev["developer"].value_counts().keys())
plt.title("Top 5 des développeurs de jeux vidéos")
plt.xticks(rotation = 90)
plt.show()

### Evolutions de leurs nombre de jeux développés au cours du temps

In [ ]:
tab_dev = donnees[(donnees["developer"] == "Capcom") | (donnees["developer"] == "Telltale Games") | (donnees["developer"] == "Konami") ]
df_dev = tab_dev.groupby(["developer", "year"]).size().reset_index(name = "Count")

In [ ]:
plt.figure(figsize=[15, 6])
plt.subplot(111)
sns.pointplot(x = "year", y = "Count", data = df_dev, hue = "developer")
plt.title("Evolution du nombre de jeux videos proposés par les 3 plus grosses sociétés de développement")
plt.xticks(rotation = 90)
plt.show()

#### Telltale Games a été fondé en Juillet 2004 et n'a fait qu'accroite sa production de Jeux Vidéos jusqu'en 2017 ou sa productivité à nettement chuté dû au fait que l'entreprise a licencié en masse pour mettre clé sous la porte.

### Genre des jeux vidéos

In [ ]:
data_genre = donnees["genre(s)"].value_counts().head()
df_genre = donnees[donnees.isin(data_genre.index)]

In [ ]:
palette = sns.color_palette("Paired")
sns.set_palette(palette)
plt.figure(figsize=[15, 6])
plt.subplot(111)
sns.countplot(data = df_genre, x = "genre(s)", order = df_genre["genre(s)"].value_counts().keys())
plt.title("Top 5 des genres de jeu vidéos")
plt.xticks(rotation = 90)
plt.show()

#### Les jeux d'actions se démarque énormément des autres genres.

In [ ]:
df_evo_genre = donnees[(donnees["genre(s)"] == "Action") | (donnees["genre(s)"] == "Action Adventure") | (donnees["genre(s)"] == "Sports") | (donnees["genre(s)"] == "Role-Playing") | (donnees["genre(s)"] == "Strategy")]
df_evo_gb = df_evo_genre.groupby(["year", "genre(s)"]).size().reset_index(name="Count")

In [ ]:
plt.figure(figsize=[15, 6])
plt.subplot(111)
sns.pointplot(x = "year", y = "Count", data = df_evo_gb, hue = "genre(s)")
plt.title("Evolution des genres de jeux videos au cours du temps")
plt.xticks(rotation = 90)
plt.show()

#### Les Jeux d'actions ont toujours été, depuis 1995, les genres les plus prisés des Videos Gamers.
#### Les jeux de sports ont, un temps, été à la seconde position dans les années 2000 notamment avec le fameux ISS PRO EVOLUTION

### Distribution des notes en ligne (metascore)

In [ ]:
palette = sns.color_palette(["#3498db", "#e74c3c"])
sns.set_palette(palette)
plt.figure()
plt.subplot(111)
sns.distplot(donnees["metascore"])
plt.title("Distribution des notes en lignes")
plt.xticks(rotation = 90)
plt.show()

#### Le pic le plus haut est compris dans le bin représentant les notes comprise entre 70 et 75

### Visualiser la relation entre score et metascore

In [ ]:
# je retire les lignes marquée "tbd"
test = donnees[donnees["user_score"] != "tbd"]
donnees = test
donnees["user_score"] = pd.to_numeric(donnees["user_score"], downcast = "float")
donnees.dtypes

In [ ]:
sns.set(style = "white", color_codes = True)
sns.jointplot(data = donnees, x = "metascore", y = "user_score", kind = "reg", line_kws = {"color" : "red"})
plt.show()

#### Les notes metascore et des joueurs sont aux alentours de 75 / 80
#### Remarque : Thanks to Michael Can and his notebook presentation where i found precious information

In [ ]:
donnees_heatmap = donnees[["genre(s)", "year", "metascore", "user_score", "platform", "publisher"]]
donnees_heatmap["genre(s)"] = donnees_heatmap["genre(s)"].astype("category").cat.codes
donnees_heatmap["year"] = donnees_heatmap["year"].astype("category").cat.codes
donnees_heatmap["metascore"] = donnees_heatmap["metascore"].astype("category").cat.codes
donnees_heatmap["user_score"] = donnees_heatmap["user_score"].astype("category").cat.codes
donnees_heatmap["publisher"] = donnees_heatmap["publisher"].astype("category").cat.codes
donnees_heatmap["platform"] = donnees_heatmap["platform"].astype("category").cat.codes
correlation = donnees_heatmap.corr()
sns.heatmap(correlation, annot = True, linewidths = 1.0)

#### Il n'y a pas vraiment de lien entre les variables si ce n'est un bout de corrélation entre les score joueurs et metascore